In [ ]:
!pip install Flask pyngrok line-bot-sdk requests --quiet
!pip install google-genai --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 818.9/818.9 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.6/165.6 kB 6.5 MB/s eta 0:00:00


In [ ]:
from google.colab import userdata

ngrok_authtoken = userdata.get('NGROK_AUTHTOKEN')
line_channel_access_token = userdata.get('LINE_CHANNEL_ACCESS_TOKEN')
line_channel_secret = userdata.get('LINE_CHANNEL_SECRET')
gemini_api_key = userdata.get('GEMINI_API_KEY')
port = 5051


In [ ]:
import os
from pyngrok import ngrok

In [ ]:
ngrok.kill()

In [ ]:
import requests
from pyngrok import ngrok

ngrok.kill() # Terminate any existing ngrok tunnels
ngrok.set_auth_token(ngrok_authtoken)
tunnel = ngrok.connect(5051, name="linebot_tunnel")
webhook_url = tunnel.public_url

print(f"Ngrok URL: {webhook_url}")

# 自動更新 LINE Webhook URL
def update_line_webhook(webhook_url):
    """使用 LINE Messaging API 更新 Webhook URL"""
    url = "https://api.line.me/v2/bot/channel/webhook/endpoint"
    headers = {
        "Authorization": f"Bearer {line_channel_access_token}",
        "Content-Type": "application/json"
    }
    data = {
        "endpoint": webhook_url
    }

    response = requests.put(url, headers=headers, json=data)

    if response.status_code == 200:
        print(f"✅ LINE Webhook URL 已自動更新為：{webhook_url}")
        return True
    else:
        print(f"❌ 更新失敗：{response.status_code} - {response.text}")
        return False

# 執行更新
update_line_webhook(webhook_url)

Ngrok URL: https://unauthenticated-marva-wisely.ngrok-free.dev
✅ LINE Webhook URL 已自動更新為：https://unauthenticated-marva-wisely.ngrok-free.dev


True

In [ ]:
from google import genai
from google.genai.types import Tool, GenerateContentConfig, GoogleSearch

# === 初始化 Google Gemini ===
client = genai.Client(api_key=gemini_api_key)

chat = client.chats.create(
    model="gemini-2.5-flash",
    config=GenerateContentConfig(
        response_modalities=["TEXT"],
    )
)

In [ ]:
def stateful_query(payload):
    response = chat.send_message(message=payload)
    return response.text

In [ ]:
result = stateful_query("簡介明新科技大學")
print(result)

明新科技大學 (Ming Hsin University of Science and Technology)，簡稱**明新科大**，是一所位於臺灣新竹縣新豐鄉的私立科技大學。

以下是明新科技大學的簡要介紹：

1.  **創校歷史與發展：**
    *   創立於**1966年**，前身為「明新工業專科學校」。
    *   歷經明新技術學院，並於**2002年**改制升格為「明新科技大學」。
    *   至今已逾半世紀，是北臺灣歷史悠久的科技大學之一。

2.  **地理位置與特色：**
    *   位於新竹縣，緊鄰**新竹科學園區**及湖口工業區等重要產業聚落。
    *   因地利之便，與周邊的高科技產業及傳統產業建立了緊密的產學合作關係，為學生提供豐富的實習與就業機會。

3.  **辦學理念與願景：**
    *   秉持「**實用、創新、卓越**」的辦學理念。
    *   致力於培育具備全球視野、在地關懷、實作能力與創新精神的專業人才。
    *   強調理論與實務結合，提升學生的就業競爭力。

4.  **學院與學系：**
    *   學校設有四大專業學院，提供學士班與碩士班課程：
        *   **工程學院：** 涵蓋電機、電子、機械、化學工程等相關領域。
        *   **管理學院：** 包含企業管理、資訊管理、財務金融、觀光休閒等科系。
        *   **服務產業學院：** 注重餐飲、旅館、幼兒保育、老人服務等服務產業人才培育。
        *   **設計學院：** 設有設計相關學系，如多媒體與遊戲設計、室內設計等。

5.  **教育特色：**
    *   **強調產學合作：** 與業界緊密結合，提供實務專題、校外實習、建教合作等機會。
    *   **注重實作能力：** 課程設計強調動手實作，培養學生的即戰力。
    *   **優良的就業率：** 畢業生因具備扎實的專業技能與豐富的實務經驗，在就業市場上表現亮眼。
    *   **多元學習環境：** 積極推動國際交流，鼓勵學生參與各類社團活動，拓展視野。

總體而言，明新科技大學是一所以技職教育為導向，強調實務應用與產業連結，在新竹地區乃至全國科技產業人才培育上扮演重要角色的大學。


In [ ]:
result2 = stateful_query("校長是誰？")
print(result2)

明新科技大學現任校長是 **劉國偉** 教授。

他於2023年8月1日就任。


In [ ]:
from flask import Flask, request, abort

from linebot.v3 import (
    WebhookHandler
)
from linebot.v3.exceptions import (
    InvalidSignatureError
)
from linebot.v3.messaging import (
    Configuration,
    ApiClient,
    MessagingApi,
    ReplyMessageRequest,
    TextMessage,
)
from linebot.v3.webhooks import (
    MessageEvent,
    TextMessageContent,
)

app = Flask(__name__)

configuration = Configuration(access_token=line_channel_access_token)
handler = WebhookHandler(line_channel_secret)


@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    print("BODY: ", body)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        app.logger.info("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'


@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    text = event.message.text
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        if text.startswith('AI '):
            prompt = text[3:]
            reply_text = stateful_query(prompt)
            line_bot_api.reply_message_with_http_info(
                ReplyMessageRequest(
                    reply_token=event.reply_token,
                    messages=[TextMessage(text=reply_text)]
                )
            )

        else:
            line_bot_api.reply_message_with_http_info(
                ReplyMessageRequest(
                    reply_token=event.reply_token,
                    messages=[TextMessage(text=event.message.text),
                        TextMessage(text=event.message.text)]
                )
            )

if __name__ == "__main__":
    app.run(port=port)

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5051
INFO:werkzeug:Press CTRL+C to quit
